In [28]:
import numpy as np

In [29]:
#загрузка массива данных 
close = np.load('close.npy')
# array([ 76.3 ,  76.15,  76.26, ...,  91.9 ,  91.82,  91.89])
time = np.load('time.npy', encoding='bytes')
N = 1000
#число рассматриваемых событий

# array([datetime.datetime(2016, 9, 1, 10, 0), ...], dtype=object)

In [30]:
#вновь поступившее событие, характеристики -- время, цена закрытия
class event():
    time = time[0]
    close = close[0]
    hour = 0
    day = 0

In [31]:
#работа со временем
t1 = time[0]
t2 = time[0]

In [32]:
#массив рассматриваемых событий
ev = event()
ev.time = time[0]
ev.close = close[0]
ev.hour = ev.time.hour
ev.day = ev.time.day
main_array = [ev for i in range(0,N)]

In [33]:
#Вспомогательные функции

#средняя цена закрытия в час для k событий, для конкретно выбранного события с номером i в массиве
def houraverageclose (i, k):
    sum = 0
    num = 0
    for j in (0, k-1):
        if main_array[j].hour == main_array[i].hour:
            sum+= main_array[j].close
            ++num
    return sum/num
    
#cредняя цена закрытия в день для k событий
def dayaverageclose (i, k):
    sum = 0
    num = 0
    for j in (0, k-1):
        if main_array[j].day == main_array[i].day:
            sum+= main_array[j].close
            ++num
    return sum/num


#количество прошедших часов 
def hnum(number_of_time_periods):
    ans = 1
    for i in range (1, number_of_time_periods-1):
        if main_array[i].hour != main_array[i-1].hour:
            ++ans
    return ans

def daynum(number_of_time_periods):
    ans = 1
    for i in range (1, number_of_time_periods-1):
        if main_array[i].day != main_array[i-1].day:
            ++ans
    return ans

def day(i):
    return main_array[i].day

def hour(i):
    return main_array[i].hour

averagehourarray = [houraverageclose(0, N) for j in range (0,hnum(N)-1)]
averagedayarray = [dayaverageclose(0, N) for j in range (0,daynum(N)-1)]
for j in range(1,N-1):
    j1 = 0
    j2 = 0
    if main_array[j].hour != main_array[j-1].hour:
        ++j1
        averagehourarray[j1] = houraverageclose(j,N-1)
    if main_array[j].day != main_array[j-1].day:
        ++j2
        averagedayarray[j2] = dayaverageclose(j,N-1)

In [34]:
#блок реализации определенной стратегии
def alpha_hours(number_of_time_periods):
    return 2/(hnum(number_of_time_periods)+1)

def alpha_days(number_of_time_periods):
    return 2/(daynum(number_of_time_periods)+1)

def ema_hour(i, k):
    if (i == 0):
        return 0
    else:
        return alpha_hours(k)*averagehourarray[i].close+(1-alpha_hours)*averagehourarray[i-1]

def dma_hour(i,k):
    if (i == 0):
        return 0
    else:
        return alpha_hours(k)*ema_hour(i,k)+(1-alpha_hours(k))*dma_hour(i-1,k)

def tma_hour(i,k):
    if (i == 0):
        return 0
    else:
        return alpha_hours(k)*dma_hour(i,k)+(1-alpha_hours(k))*tma_hour(i-1,k)

def trix_hour(i,k):
    if (i==0):
        return 0
    else:
        return (tma_hour(i,k) - tma_hour(i-1,k))/tma_hour(i-1,k)*100

def ema_day(i,k):
    if (i == 0):
        return 0
    else:
        return alpha_days(k)*averagedayarray[i].close+(1-alpha_days(k))*averagedayarray[i-1]

def dma_day(i,k):
    if (i == 0):
        return 0
    else:
        return alpha_days(k)*ema_day(i,k)+(1-alpha_days(k))*dma_day(i-1,k)

def tma_day(i,k):
    if (i == 0):
        return 0
    else:
        return alpha_days(k)*dma_day(i,k)+(1-alpha_days(k))*tma_day(i-1,k)

def trix_day(i,k):
    if (i==0):
        return 0
    else:
        return (tma_day(i,k) - tma_day(i-1,k))/tma_day(i-1,k)*100 

#стратегия Trix принятия решений в момент времени i
def strategy (i):
    if ((trix_day(day(i-2),i-1) > trix_day(day(i-1),i-1) & (trix(hour(i-2),i-1) < trix(hour(i-1),i-1)))):
        print main_array[i].time, "buy"
        return 1
    elif (trix_day(day(i-2),i-1) > trix_day(day(i-1),i-1)) & (trix(hour(i-2),i-1) >= trix(hour(i-1),i-1)):
        print main_array[i].time,"close position"
        return 0
    elif (trix_day(day(i-2),i-1) < trix_day(day(i-1),i-1)) & (trix(hour(i-2),i-1) < trix(hour(i-1),i-1)):
        print main_array[i].time,"close position"
        return 0
    elif (trix_day(day(i-2),i-1) > trix_day(day(i-1),i-1)) & (trix(hour(i-2),i-1) >= trix(hour(i-1),i-1)):
        print main_array[i].time,"sell"
        return -1

In [35]:
#инициализация произвольной стратегии на массиве событий
def strategy_init(k):
    ev = event()
    for i in (0, k-1):
        ev.time= time[i];
        ev.close = close[i]
        ev.hour = time[i].hour
        ev.day = time[i].day
        main_array[i] = ev
    return strategy(k)

In [36]:
#основной цикл программы
def main_cycle():
    profit = 0
    buy = 0
    i = 20
    res = strategy_init(i)
    while i < N:
        if (buy == 1) and (res== 1):
            profit += main_array[i].close - main_array[i-1].close
        elif (buy == 1) and (res == 0):
            buy = 0
        elif (buy == 1) and (res == -1):
            buy = -1
            profit += -main_array[i].close + main_array[i-1].close
        elif (buy == -1) and (res == 1):
            buy = 1
            profit += main_array[i].close - main_array[i-1].close
        elif (buy == -1 ) and (res == -1):
            profit += -main_array[i].close + main_array[i-1].close
        elif (buy == -1) and (res == 0):
            buy = 0
        elif (buy == 0) and (res == 0):
            buy = 0
        elif (buy == 0) and (res == 1):
            buy = 1
            profit += main_array[i].close - main_array[i-1].close
        elif (buy == 0) and  (res == -1):
            buy = -1
            profit += -main_array[i].close + main_array[i-1].close
        ++i
    return profit

In [37]:
main_cycle()

IndexError: list index out of range